## Squeeze-and-Excitation Networks (SENet)


## **What “SE” Stands For**

**SE block** = **Squeeze-and-Excitation block**.
It’s a lightweight attention mechanism for CNNs introduced in the paper:

> *“Squeeze-and-Excitation Networks” (Hu et al., CVPR 2018)*

It improves a network’s ability to model the **importance of each channel** in feature maps.


<img src="images/squeeze_and_excitation_networks_SENet.png" />

---

#### **Why We Need It**

Convolutions learn spatial filters but treat all channels equally.
Some channels might carry more relevant information for the current task.
An SE block lets the network **recalibrate channel-wise feature responses** adaptively.

---

#### **How an SE Block Works**

Let’s say the input feature map is $X\in \mathbb{R}^{H\times W\times C}$ (height, width, channels).

#### Step A: **Squeeze** (Global information)

* Perform **global average pooling** over spatial dimensions $H \times W$ to get one value per channel.
* This yields a vector $z\in \mathbb{R}^{C}$ summarizing each channel’s global response.

Mathematically:
$$
z_c = \frac{1}{H , W}\sum_{i=1}^{H}\sum_{j=1}^{W} X_{i,j,c}
$$

#### Step B: **Excitation** (Learn channel attention)

* Pass (z) through a small **two-layer MLP**:

  * First layer reduces dimension to $C/r$ (bottleneck; $r$ is reduction ratio like 16)
  * Apply ReLU.
  * Second layer expands back to $C$.
  * Apply sigmoid to get weights $s\in[0,1]^C$.

$$
s = \sigma(W_2 , \delta(W_1 z))
$$

where $\delta$ is ReLU, $\sigma$ is sigmoid.

#### Step C: **Scale** (Recalibration)

* Multiply the original feature map channels by the learned weights:

$$
\tilde{X}{i,j,c} = s_c \cdot X{i,j,c}
$$

So channels the block deems “important” get boosted; less important channels get suppressed.

---

#### **Visual Diagram**

```
Input Feature Map (H×W×C)
       │
 Global Average Pooling (Squeeze)
       ↓
 Channel Descriptor (C)
       │
 Fully Connected (reduce C→C/r), ReLU
       │
 Fully Connected (C/r→C), Sigmoid (Excitation)
       ↓
 Channel Weights (C)
       │
 Scale original feature map channel-wise
       ↓
Output Feature Map (H×W×C)
```

---

#### **Where It’s Used**

* Originally introduced in **SENet** (ImageNet winner 2017/2018).
* Incorporated into **MobileNetV3**, **EfficientNet** (in MBConv blocks), ResNeXt, etc.
* Adds only a tiny computational overhead but often improves accuracy significantly.

---



## **Numerical Eexample**


Assume we have a small **feature map** from some CNN layer:

$$
X \in \mathbb{R}^{4\times4\times3}
$$
So:

* Height = 4
* Width = 4
* Channels = 3

We’ll pick a **reduction ratio** $r = 3$ → bottleneck dimension = $C / r = 1$.

---

####  **Squeeze** (Global Average Pooling)

Compute one average per channel:

$$
z_c = \frac{1}{H \times W} \sum_{i,j} X_{i,j,c}
$$

Let’s assume the averages come out as:
$$
z = [2.0, 0.5, 1.0]
$$

So now we have a vector of size (3).

---

#### **Excitation** (Two small fully-connected layers)

**First FC layer (reduce channels)**

$$
W_1: \mathbb{R}^{3 \rightarrow 1}
$$

We’ll use these weights for illustration:
$$
W_1 = [0.2, 0.4, 0.1]
$$

Compute:
$$
h = W_1 \cdot z = 0.2(2.0) + 0.4(0.5) + 0.1(1.0) = 0.4 + 0.2 + 0.1 = 0.7
$$

Apply ReLU:
$$
h = \max(0, 0.7) = 0.7
$$

---

**Second FC layer (expand back)**

$$
W_2: \mathbb{R}^{1 \rightarrow 3}
$$

Say $W_2 = [0.5, 1.0, -0.5]^T$

Compute:
$$
s = W_2 \cdot h = [0.5×0.7, 1.0×0.7, -0.5×0.7] = [0.35, 0.7, -0.35]
$$

Apply **sigmoid** to get weights between 0 and 1:
$$
\sigma(s) = [0.586, 0.668, 0.413]
$$

These are our **channel attention weights**.

---

#### **Scale** (Channel Recalibration)

Now, multiply each channel of the original feature map by its corresponding weight:

| Channel | Scale weight | Example original mean | Scaled mean        |
| ------- | ------------ | --------------------- | ------------------ |
| 1       | 0.586        | 2.0                   | 2.0 × 0.586 = 1.17 |
| 2       | 0.668        | 0.5                   | 0.5 × 0.668 = 0.33 |
| 3       | 0.413        | 1.0                   | 1.0 × 0.413 = 0.41 |

→ The first channel gets boosted moderately,
→ The second a bit,
→ The third is suppressed.

---

#### Intuition

* The SE block **“squeezed”** spatial info into 3 global averages.
* Then it **“excited”** them with a small MLP to learn *which channels matter*.
* Finally, it **scaled** the original channels by those learned importance weights.

So the network learns to emphasize informative channels and dampen unhelpful ones — dynamically, per input image.

---




#### **PyTorch Implementation (simple)**

```python
import torch
import torch.nn as nn

class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)        # Squeeze
        y = self.fc(y).view(b, c, 1, 1)        # Excitation
        return x * y                          # Scale
```

---



## **Spatial Attention vs Spatial Attention**
####  1. The Core Difference

| Type                            | Focus                 | What it learns                         | Output shape  |
| ------------------------------- | --------------------- | -------------------------------------- | ------------- |
| **Squeeze-and-Excitation (SE)** | **Channel attention** | How important each *channel* is        | ( (1, 1, C) ) |
| **Spatial Attention**           | **Spatial attention** | How important each *pixel/location* is | ( (H, W, 1) ) |

In short:

* **SE → “Which feature maps (channels) should I amplify?”**
* **Spatial → “Which spatial regions (pixels) should I focus on?”**

---

####  **2. Mechanism Breakdown**

A. **Squeeze-and-Excitation (Channel Attention)**

1. **Squeeze:** Global average pooling → summarize each channel → vector of length (C).
2. **Excite:** MLP → outputs one weight per channel (values 0–1).
3. **Scale:** Multiply each channel globally by its weight.

  **Effect:** Emphasizes useful *feature types* (e.g., “edges,” “textures,” “object color” channels).
Every pixel within a channel gets scaled equally.

**Visualization:**

```
Input:  H×W×C
 ↓ (Global Average Pool)
Vector (1×1×C)
 ↓ (2 FC layers + sigmoid)
Weights (1×1×C)
 ↓ (channel-wise multiply)
Output: H×W×C
```

---

B. **Spatial Attention**

1. **Squeeze channels:** Compute an importance map for *each spatial location* instead of each channel.
   Typically:

   * Apply average pooling and max pooling along channels → two 2D maps (H×W×1 each).
2. **Concatenate them** and run a small 2D convolution (e.g., 7×7).
3. **Sigmoid** → get spatial attention map (H×W×1).
4. **Multiply** original feature map spatially.

   **Effect:** Emphasizes *where* in the image to focus (e.g., object regions vs. background).
Each pixel gets a unique weight.

**Visualization:**

```
Input:  H×W×C
 ↓ (Channel AvgPool + MaxPool → concat)
Feature (H×W×2)
 ↓ (Conv 7×7 + sigmoid)
Attention Map (H×W×1)
 ↓ (spatial multiply)
Output: H×W×C
```

---

####  3. Comparison Table

| Aspect             | Squeeze-and-Excitation (SE)           | Spatial Attention                      |
| ------------------ | ------------------------------------- | -------------------------------------- |
| Focus              | Channels                              | Spatial positions                      |
| Key question       | *“Which feature maps are important?”* | *“Which image regions are important?”* |
| Pooling used       | Global average pooling (over H, W)    | Pool over channels (avg & max)         |
| Weight shape       | (1×1×C)                               | (H×W×1)                                |
| Computational cost | Very low                              | Slightly higher (7×7 conv)             |
| Used in            | EfficientNet, SENet, MobileNetV3      | CBAM, BAM, attention-based CNNs        |
| Effect             | Reweight feature *types*              | Reweight feature *locations*           |

---

####  4. Combining Both (CBAM)

Many modern networks (e.g., **CBAM — Convolutional Block Attention Module**) use *both*:

1. Apply SE (channel attention).
2. Then apply spatial attention.

This way the network learns **what** and **where** to focus.

---

####  5. Intuitive Analogy

| Analogy           | Meaning                                                         |
| ----------------- | --------------------------------------------------------------- |
| SE block          | “Turn up the volume for the *important instruments* (channels)” |
| Spatial attention | “Focus your *eyes* on where the action happens in the image”    |

Together → you both **listen carefully** and **look carefully** 👀🎧

---

Would you like me to show a **small visual example (with a heatmap)** comparing how SE and spatial attention would reweight an example 3×3×3 feature map?
